In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline

In [2]:
#Required text pre-processing libraries are imported
import string
import nltk
import re

# download the stopwords and wordnet corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
# import tokenize from nltk library
from nltk import tokenize
# import WordNetLemmatizer from nltk library
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from nltk.probability import FreqDist

#Required data visualisation libraries are imported
import plotly.express as px
import seaborn as sns 
import matplotlib.pyplot as plt


#Required prediction modelling libraries are imported
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import plot_confusion_matrix, confusion_matrix, precision_recall_curve, auc, roc_curve, accuracy_score, recall_score, classification_report, f1_score, precision_score, precision_recall_fscore_support, roc_auc_score, make_scorer
from sklearn.pipeline import Pipeline

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

import xgboost
from xgboost import XGBClassifier


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jillian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jillian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jillian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jillian/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
tweets=pd.read_csv('tweets.csv')

In [4]:
tweets.head()

,849670936924172289,2022-08-08 23:59:43+00:00,,1556792300998639618,en,1,0,0.1,0.2,Twitter for iPhone,same! she dresses like rich me would 😭😍 https://t.co/adQU7hKk1w
0,1174554679822307329,2022-08-08 23:59:30+00:00,,1556792249106644992,en,1,0,0,1,Twitter for iPhone,@LionBrosPodcast Summer dresses
1,1236458791962644480,2022-08-08 23:59:22+00:00,,1556792215120265217,en,2,0,1,0,Twitter for Android,"one direction, all time low, 5sos, the hunger ..."
2,1306498176854175744,2022-08-08 23:59:19+00:00,,1556792202524872704,en,0,0,0,0,Twitter for iPhone,I bought 2 dresses for 1 gala bc I just couldn...
3,1357815347991183366,2022-08-08 23:58:34+00:00,,1556792011734261763,en,0,0,0,0,Twitter for iPhone,@gabrielafotoo @EmmetCohen He always dresses s...
4,1376947999901937671,2022-08-08 23:58:29+00:00,,1556791994327781377,en,0,0,0,0,Poshmark,Check out this listing I just added to my #Pos...


In [7]:
# Access column names
tweets.columns

Index(['849670936924172289', '2022-08-08 23:59:43+00:00', ' ',
       '1556792300998639618', 'en', '1', '0', '0.1', '0.2',
       'Twitter for iPhone',
       'same! she dresses like rich me would 😭😍 https://t.co/adQU7hKk1w'],
      dtype='object')

In [13]:
# Rename columns
tweets.columns = ['author_id', 'created_at', 'geo_location', 'tweet_id', 'lang', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'source', 'text']

In [14]:
tweets.head()

,author_id,created_at,geo_location,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,text
0,1174554679822307329,2022-08-08 23:59:30+00:00,,1556792249106644992,en,1,0,0,1,Twitter for iPhone,@LionBrosPodcast Summer dresses
1,1236458791962644480,2022-08-08 23:59:22+00:00,,1556792215120265217,en,2,0,1,0,Twitter for Android,"one direction, all time low, 5sos, the hunger ..."
2,1306498176854175744,2022-08-08 23:59:19+00:00,,1556792202524872704,en,0,0,0,0,Twitter for iPhone,I bought 2 dresses for 1 gala bc I just couldn...
3,1357815347991183366,2022-08-08 23:58:34+00:00,,1556792011734261763,en,0,0,0,0,Twitter for iPhone,@gabrielafotoo @EmmetCohen He always dresses s...
4,1376947999901937671,2022-08-08 23:58:29+00:00,,1556791994327781377,en,0,0,0,0,Poshmark,Check out this listing I just added to my #Pos...


In [15]:

import numpy as np
import pandas as pd
import json
import pickle
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

class ReviewClassifierService:

    words = {}

    model = pickle.load(open('reviewClassifier.pkl','rb'))

    def __init__(self):
        with open('word_feature_space.json', 'r') as fp:
            self.words = json.load(fp)

    #clean review get word roots, lowercase, split them by words and etc.
    def clean_review(self, review):
        review = review.lower()
        review = review.split()

        ps = PorterStemmer()

        review = [ps.stem(word) for word in review
                    if not word in set(stopwords.words('english'))]

        review = ' '.join(review)

        return review

    #return if the review was positive or negative as JSON.
    def classify(self, review):
        length = len(self.words)
        incoming = [0] * length
        clean_sample = self.clean_review(review)
        clean_words = re.sub("[^\w]", " ", clean_sample).split()

        for i in clean_words:
            if i in self.words:
                index = self.words[i]
                incoming[index] += 1

        outcome = self.model.predict([incoming])

        result = {'outcome': int(outcome[0])}

        return json.dumps(result)

FileNotFoundError: [Errno 2] No such file or directory: 'reviewClassifier.pkl'